In [9]:
import csv
data_path = './DATA/korean_smile_style_dataset/smilestyle_dataset.tsv'
f= open(data_path,'r',encoding='utf-8')
rdr = csv.reader(f, delimiter='\t' )

for line in rdr:
    print(line)
    break

['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']


In [5]:
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session)
    return final_data

In [37]:
import csv

data_path = './DATA/korean_smile_style_dataset/smilestyle_dataset.tsv'
f= open(data_path,'r',encoding='utf-8')
rdr = csv.reader(f, delimiter='\t')

session_dataset = []
session = []


for i, line in enumerate(rdr):
    if i == 0:
        header = line
    else:
        utt = line[0] # format
        if utt.strip() != '':
            session.append(utt)
        else:
            session_dataset.append(session)
            session = []
'''마지막 세션 저장'''
session_dataset.append(session)
f.close()

In [38]:
session_dataset[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

In [39]:
session_dataset[-1]

['안녕하세요, 잘 지내고 있으세요?',
 '사실 약간 우울한 것 같아요. 그쪽은 잘 지내고 있으세요?',
 '어째서 우울하세요?',
 '제가 제일 좋아하는 티비쇼가 꽤 오랫동안 안 나올 것 같아서 슬프네요.',
 '게임 오브 쓰론 말씀하세요?',
 '네, 2019년에서야 다시 찍는다고 발표했어요.',
 '그래도 아예 멈춘 것보다는 나은 것 같아요.',
 '네, 이야기를 매듭지 않고 끝냈으면 정말 슬펐을 것 같아요.',
 '그럼 이제는 어떤 쇼를 볼 건가요?',
 '블랙 미러도 재미있다고 들었는데, 어떤가요?',
 '저는 티비를 자주 안 봐서 사실 잘 몰라요.',
 '그러면 직접 알아봐야겠네요.',
 '새로운 티비쇼를 개척하는 것도 재밌을거에요.',
 '저도 기대하고 있습니다.']

In [40]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [41]:
# dir(tokenizer)
tokenizer.special_tokens_map
'''eos와 sep토큰이 달라야 논문데로 구현'''

'eos와 sep토큰이 달라야 논문데로 구현'

In [42]:
tokenizer.eos_token, tokenizer.sep_token, tokenizer.eos_token_id, tokenizer.sep_token_id

('[SEP]', '[SEP]', 2, 2)

In [43]:
special_tokens = {'sep_token':'<SEP>'}
tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))
'''모델이 학습한 데이터구조가 달라졌으므로 적용'''

'모델이 학습한 데이터구조가 달라졌으므로 적용'

In [44]:
tokenizer.eos_token, tokenizer.sep_token, tokenizer.eos_token_id, tokenizer.sep_token_id

('[SEP]', '<SEP>', 2, 32000)

In [45]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 4)

MLM 구조

In [46]:
'''MLM (masked language model )마스크 된 부분을 예측하는 것'''
import random

session = session_dataset[0]
mask_ratio = 0.15
corrupt_tokens = []
output_tokens = []
for i, utt in enumerate(session):
    original_token = tokenizer.encode(utt, add_special_tokens=False)

    mask_num = int(len(original_token)*mask_ratio)
    mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
    corrupt_token = []
    for pos in range(len(original_token)):
        if pos in mask_positions:
            corrupt_token.append(tokenizer.mask_token_id)
        else:
            corrupt_token.append(original_token[pos])
    
    if i == len(session)-1:
        output_tokens += original_token
        corrupt_tokens += corrupt_token
    else:
        output_tokens += original_token + [tokenizer.sep_token_id]
        corrupt_tokens += corrupt_token + [tokenizer.sep_token_id]

In [47]:
print(tokenizer.decode(output_tokens))
print(tokenizer.decode(corrupt_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다. 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나요? <SEP> 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.
안녕하세요. 저는 고양이 6마리 [MASK]요. <SEP> [MASK]를 6마리나 [MASK]? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 [MASK]게 [MASK]진 않아요. <SEP> 가장 나이가 [MASK]은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요 [MASK] <SEP> 그럼 가장 어린 [MASK]가 어떻게 돼요? <SEP> 한 살 [MASK]. 작년에 분양 받았어요. <SEP> 그럼 [MASK]들끼리 안 싸우나요? <SEP> 저희 일곱은 [MASK]같이 한 가족입니다 [MASK] 싸우는 일은 없어요.


URC를 위한 입력

In [48]:
'''short session'''
k = 4 # 논문에서 3개는 context 1개는 response가 가장 성능 좋음
short_session_dataset = []
for session in session_dataset:
    for i in range(len(session)-k+1):
        short_session_dataset.append(session[i:i+k])

print(short_session_dataset[0])

['안녕하세요. 저는 고양이 6마리 키워요.', '고양이를 6마리나요? 키우는거 안 힘드세요?', '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.', '가장 나이가 많은 고양이가 어떻게 돼요?']


In [49]:
import random

all_utts = set()
for session in session_dataset:
    for utt in session:
        all_utts.add(utt)

all_utts = list(all_utts)
print(len(all_utts), all_utts[0])

3430 안녕하세요, 지금까지 저녁은 잘 즐기고 계신가요?


In [52]:
import random

session = short_session_dataset[0]
urc_tokens = []
context_utts = []
for i in range(len(session)):
    utt = session[i]
    original_token = tokenizer.encode(utt, add_special_tokens=False)

    if i == len(session)-1:
        positive_token = urc_tokens + original_token
        while True:
            random_neg_response = random.choice(all_utts)
            if random_neg_response not in context_utts:
                break
        random_neg_response_token = tokenizer.encode(random_neg_response, add_special_tokens=False)
        random_token = urc_tokens + random_neg_response_token
        ''' context nag rep 입력'''
        context_neg_response = random.choice(context_utts)
        context_neg_response_token = tokenizer.encode(context_neg_response, add_special_tokens=False)
        context_neg_tokens = urc_tokens + context_neg_response_token
    else:
        urc_tokens += original_token + [tokenizer.sep_token_id]
    context_utts.append(utt)
    

In [53]:
print(tokenizer.decode(positive_token))
print('#######')
print(tokenizer.decode(random_token))
print('#######')
print(tokenizer.decode(context_neg_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
#######
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 대단하시네요. 체스 동아리에는 사람이 많이 있나요?
#######
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 안녕하세요. 저는 고양이 6마리 키워요.


post train 데이터 로더

In [61]:
import csv
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

class post_loader(Dataset):
    def __init__(self, data_path) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token':'<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)

        f = open(data_path, 'r', encoding='utf-8')
        rdr = csv.reader(f, delimiter='\t')

        session_dataset = []
        session = []


        for i, line in enumerate(rdr):
            if i == 0:
                header = line
            else:
                utt = line[0] # format
                if utt.strip() != '':
                    session.append(utt)
                else:
                    session_dataset.append(session)
                    session = []
        '''마지막 세션 저장'''
        session_dataset.append(session)
        f.close()

        '''short session context'''
        k = 4 # 논문에서 3개는 context 1개는 response가 가장 성능 좋음
        self.short_session_dataset = []
        for session in session_dataset:
            for i in range(len(session)-k+1):
                self.short_session_dataset.append(session[i:i+k])
        
        '''모든 발화 저장'''
        self.all_utts = set()
        for session in session_dataset:
            for utt in session:
                self.all_utts.add(utt)

        self.all_utts = list(self.all_utts)
    
    def __len__(self):
        return len(self.short_session_dataset)

    def __getitem__(self, idx) :
        session = self.short_session_dataset[idx]
        '''MLM'''
        mask_ratio = 0.15
        self.corrupt_tokens = []
        self.output_tokens = []
        for i, utt in enumerate(session):
            original_token = tokenizer.encode(utt, add_special_tokens=False)

            mask_num = int(len(original_token)*mask_ratio)
            mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
            corrupt_token = []
            for pos in range(len(original_token)):
                if pos in mask_positions:
                    corrupt_token.append(tokenizer.mask_token_id)
                else:
                    corrupt_token.append(original_token[pos])
            
            if i == len(session)-1:
                self.output_tokens += original_token
                self.corrupt_tokens += corrupt_token
            else:
                self.output_tokens += original_token + [tokenizer.sep_token_id]
                self.corrupt_tokens += corrupt_token + [tokenizer.sep_token_id]
        
        '''label for loss'''
        self.corrupt_mask_positions = []
        for pos in range(len(self.corrupt_tokens)):
            if self.corrupt_tokens[pos] == self.tokenizer.mask_token_id:
                self.corrupt_mask_positions.append(pos)
        
        '''URC'''
        urc_tokens = []
        context_utts = []
        for i in range(len(session)):
            utt = session[i]
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)

            if i == len(session)-1:
                urc_tokens += [self.tokenizer.eos_token_id]
                self.positive_token = [self.tokenizer.cls_token_id] + urc_tokens + original_token
                while True:
                    random_neg_response = random.choice(all_utts)
                    if random_neg_response not in context_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(random_neg_response, add_special_tokens=False)
                self.random_token = [self.tokenizer.cls_token_id] + urc_tokens + random_neg_response_token
                ''' context nag rep 입력'''
                context_neg_response = random.choice(context_utts)
                context_neg_response_token = self.tokenizer.encode(context_neg_response, add_special_tokens=False)
                self.context_neg_tokens = [self.tokenizer.cls_token_id] + urc_tokens + context_neg_response_token
            else:
                urc_tokens += original_token + [self.tokenizer.sep_token_id]
            context_utts.append(utt)
        return self.corrupt_tokens, self.output_tokens, self.corrupt_mask_positions, [self.positive_token, self.random_token, self.context_neg_tokens], [0,1,2]

    '''배치 처리'''
    def collate_fn(self, sessions):
        batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels = [], [], [], [], []
        batch_min_attentions, batch_urc_attentions = [], []
        corrupt_max_len, urc_max_len = 0, 0

        '''MLM, URC에서 가장 긴 입력 길이 찾기'''
        for session in sessions:
            corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = session
            if len(corrupt_tokens) > corrupt_max_len:
                corrupt_max_len = len(corrupt_tokens)
            positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            if max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)]) > urc_max_len:
                urc_max_len = max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)])
        
        '''padding'''
        for session in sessions:
            corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = session
            '''min'''
            batch_corrupt_tokens.append(corrupt_tokens + [self.tokenizer.pad_token_id for _ in range(corrupt_max_len - len(corrupt_tokens))])
            batch_min_attentions.append([1 for _ in range(len(corrupt_tokens))] + [0 for _ in range(corrupt_max_len - len(corrupt_tokens))])

            batch_output_tokens.append(output_tokens + [self.tokenizer.pad_token_id for _ in range(corrupt_max_len - len(corrupt_tokens))])

            batch_corrupt_mask_positions.append(corrupt_mask_positions)
            '''urc'''
            for urc_input in urc_inputs:
                batch_urc_inputs.append(urc_input + [self.tokenizer.pad_token_id for _ in range(urc_max_len - len(urc_input))])
                batch_urc_attentions.append([1 for _ in range(len(urc_input))] + [0 for _ in range(urc_max_len - len(urc_input))])

            batch_urc_labels += urc_labels
        
        return torch.tensor(batch_corrupt_tokens), torch.tensor(batch_output_tokens), batch_corrupt_mask_positions, torch.tensor(batch_urc_inputs), \
            torch.tensor(batch_urc_labels), torch.tensor(batch_min_attentions), torch.tensor(batch_urc_attentions)

In [67]:
data_path = './DATA/korean_smile_style_dataset/smilestyle_dataset.tsv'
post_dataset = post_loader(data_path)
post_dataloader = DataLoader(post_dataset, batch_size=2, shuffle=False, collate_fn=post_dataset.collate_fn)


In [68]:
corrupt_tokens, output_tokens, corrupt_mask_positions, urc_input, urc_label = post_dataset[0]

In [69]:
print(tokenizer.decode(corrupt_tokens))
print('#######')
print(tokenizer.decode(output_tokens))
print('#######')
print(corrupt_mask_positions)
print('----------------------------')
print(tokenizer.decode(urc_input[0]))
print('#######')
print(tokenizer.decode(urc_input[1]))
print('#######')
print(tokenizer.decode(urc_input[2]))

안녕하세요. [MASK]는 고양이 6마리 키워요. <SEP> [MASK]를 6 [MASK]나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않 [MASK]요 [MASK] <SEP> 가장 나이가 많은 고양이가 [MASK] 돼요?
#######
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
#######
[4, 13, 16, 39, 41, 50]
----------------------------
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 가장 나이가 많은 고양이가 어떻게 돼요?
#######
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 안녕하세요. 추석을 잘 보내셨나요?
#######
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 고양이를 6마리나요? 키우는거 안 힘드세요?


In [70]:
for data in post_dataloader:
    print(data)
    break

(tensor([[ 5891,  2205,     4,    18,  1535,  2259,  7003,    26, 12736,  5972,
          2182,    18, 32000,  7003,     4,    26, 12736,  2075,  2182,    35,
             4,  2259,  2180,  1378, 18941,  5971,    35, 32000,  1545,  2116,
          7020,     4,  2138, 30581,  1750,  2318,     4,  2043,  1380,  2227,
          2182,    18, 32000,  3676,  4358,  2116,  1039,  2073,  7003,  2116,
          3842,     4,    35],
        [ 7003,  2138,     4, 12736,  2075,  2182,    35,  5688,     4,  2180,
          1378, 18941,  5971,    35, 32000,  1545,  2116,  7020,  7003,  2138,
             4,     4,  2318,  4390,  2043,  1380,  2227,  2182,    18, 32000,
          3676,  4358,  2116,  1039,     4,  7003,  2116,  3842,  6961,    35,
         32000,  7070,  1236, 12190,    18, 14008,  7003,     4,  2182,    18,
             1,     1,     1]]), tensor([[ 5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,  5972,
          2182,    18, 32000,  7003,  2138,    26, 12736,  2075,  

모델링

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained('klue/roberta-base')
model.resize_token_embeddings(len(tokenizer))
model.eval()
# MLM은 mask 토큰을 재구성 하는 것이기 때문에 
# 출력 size가 사전 크기여야 하는데
# automodel로 가져오면 출력 크기가 다르다
# dense layer에서 감소하기 때문

In [75]:
from transformers import RobertaForMaskedLM, AutoTokenizer
model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

special_tokens = {'sep_token':'<SEP>'}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))


Embedding(32001, 768)

In [77]:
from transformers import RobertaForMaskedLM, AutoTokenizer
import torch
import torch.nn as nn

class PostModel(nn.Module):
    def __init__(self) -> None:
        super(PostModel, self).__init__()
        self.model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
        self.hiddenDim = self.model.config.hidden_size

        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token':'<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        self.model.resize_token_embeddings(len(tokenizer))

        self.W = nn.Linear(self.hiddenDim, 3)

    def forword(self, batch_corrupt_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_min_attentions, batch_urc_attentions):
        # MLM
        corrupt_outputs = self.model(batch_corrupt_tokens, attention_mask = batch_min_attentions)['logit'] # [2, 53, 32001]
        corrupt_mask_outputs = []

        for i_batch in range(len(batch_corrupt_mask_positions)):
            corrupt_mask_output = []
            batch_corrupt_mask_position = batch_corrupt_mask_positions[i_batch]
            for pos in batch_corrupt_mask_position:
                corrupt_mask_output.append(corrupt_outputs[i_batch, pos, :].unsqueeze(0)) # [1, 32001]
            corrupt_mask_outputs.append(torch.cat(corrupt_mask_output, 0)) # [mask_num, 32001]

        # URC
        urc_outputs = self.model(batch_urc_inputs, attention_mask=batch_urc_attentions, output_hidden_states=True)['hidden_states'][-1] # [6,61,768]
        urc_logits = self.W(urc_outputs) # [6,61,3]
        urc_cls_outputs = urc_logits[:,0,:] # [6, 3]

        return corrupt_mask_outputs, urc_cls_outputs

In [1]:
from SRC.model import PostModel
post_model = PostModel().cuda()

In [2]:
import torch.nn as nn
import os

def CELoss(pred_outs, labels):
    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs,labels)
    return loss_val

def SaveModel(epoch, model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save({'epoch' : epoch, 'model' : model})

In [3]:
import torch
import torch.nn as nn
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from tqdm import tqdm
import pdb
import os

def CELoss(pred_outs, labels):
    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs,labels)
    return loss_val

def SaveModel(model_info, path, model_name):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model_info, os.path.join(path, model_name))

from SRC.dataset import post_loader
from SRC.model import PostModel

data_path = './DATA/korean_smile_style_dataset/smilestyle_dataset.tsv'
post_model = PostModel().cuda()
post_dataset = post_loader(data_path)
post_dataloader = DataLoader(post_dataset, batch_size=2, shuffle= False, collate_fn=post_dataset.collate_fn)

training_epochs = 1
max_grad_norm = 10
lr = 1e-5
num_traning_steps = len(post_dataset)*training_epochs
num_warmup_steps = len(post_dataset)
optimizer = torch.optim.AdamW(post_model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_traning_steps, num_training_steps=num_traning_steps)

for epoch in range(training_epochs):
    post_model.train()
    for i_batch, data in enumerate(tqdm (post_dataloader)):
        batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels, batch_mlm_attentions, batch_urc_attentions = data
        batch_corrupt_tokens = batch_corrupt_tokens.cuda()
        batch_output_tokens = batch_output_tokens.cuda()
        batch_urc_inputs = batch_urc_inputs.cuda()
        batch_urc_labels = batch_urc_labels.cuda()
        batch_mlm_attentions = batch_mlm_attentions.cuda()
        batch_urc_attentions = batch_urc_attentions.cuda()

        """Prediction"""
        corrupt_mask_outputs, urc_cls_outputs = post_model(batch_corrupt_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_mlm_attentions, batch_urc_attentions)
        """Loss calculation & training"""
        original_token_indexs = []
        for i_batch in range (len (batch_corrupt_mask_positions)):
            original_token_index = []
            batch_corrupt_mask_position = batch_corrupt_mask_positions[i_batch]
            for pos in batch_corrupt_mask_position:
                original_token_index.append(batch_output_tokens[i_batch, pos].item())
            original_token_indexs.append(original_token_index)
        mlm_loss = 0
        for corrupt_mask_output, original_token_index in zip(corrupt_mask_outputs, original_token_indexs):
            mlm_loss += CELoss (corrupt_mask_output, torch.tensor (original_token_index). cuda ( ) )
        urc_loss = CELoss (urc_cls_outputs, batch_urc_labels)
        loss_val= mlm_loss + urc_loss
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(post_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad ()
        model_info = {'epoch' : epoch, 'model' : post_model.state_dict(), 'optimizer' : optimizer, 'loss' : loss_val}
SaveModel(model_info, path='./MODEL/', model_name=f'PostModel_{epoch}.bin')


  0%|          | 0/1381 [00:00<?, ?it/s]


NotImplementedError: Module [PostModel] is missing the required "forward" function